# Movie Recommender

Tim Rogers

In [1]:
import pandas as pd
import numpy as np
import re
import warnings 
warnings.filterwarnings('ignore') 
  

In [ ]:
#data available at
# https://www.kaggle.com/datasets/odedgolden/movielens-1m-dataset
filepath = 'C:/Users/Tim\Documents/ml-latest-small'
link_file = filepath + '/links.csv'
movie_file = filepath + '/movies.csv'
ratings_file = filepath+'/ratings.csv'
tags_file = filepath + '/tags.csv'




In [3]:
df_link = pd.read_csv(link_file)
df_movie = pd.read_csv(movie_file)
df_ratings = pd.read_csv(ratings_file)
df_tags = pd.read_csv(tags_file)

# Merging datasets

The datasets all have a common movieID column. I therefore merged them together on that column

In [4]:
df_link.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [12]:
df_movie.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
df_ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [14]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [15]:
df_movie_tags = df_movie.merge(df_link, left_on = 'movieId', right_on = 'movieId')

In [16]:
df_movie_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   title    9742 non-null   object 
 2   genres   9742 non-null   object 
 3   imdbId   9742 non-null   int64  
 4   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 380.7+ KB


In [17]:
df_movie_more = df_movie_tags.merge(df_tags, left_on = 'movieId', right_on = 'movieId')

In [18]:
df_movie_more.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    3683 non-null   int64  
 1   title      3683 non-null   object 
 2   genres     3683 non-null   object 
 3   imdbId     3683 non-null   int64  
 4   tmdbId     3683 non-null   float64
 5   userId     3683 non-null   int64  
 6   tag        3683 non-null   object 
 7   timestamp  3683 non-null   int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 230.3+ KB


In [19]:
df_movie_all = df_movie_more.merge(df_ratings, left_on = 'movieId', right_on = 'movieId')

# Designing the Recommender

The first step of the recommender is the movie request. This is done by a simple input function

In [22]:
movie_request = str(input())

 shawshank


We now have to check the movie against the list. The function below iterates through the list of titles and finds any matches or partial matches. It appends that entry and all it's information to a list of possible movie candidates.

In [ ]:
new_list = []
for item in df_movie['title']:
    if movie_request.lower() in item.lower():
        print('found')
        new_list.append(df_movie[df_movie['title'] == item])
    else:
        print('not found')


In [170]:
#this is what the output would be for shawshank
new_list[0]['title']

277    Shawshank Redemption, The (1994)
Name: title, dtype: object

Next, using the method on https://analyticsindiamag.com/ai-mysteries/how-to-build-your-first-recommender-system-using-python-movielens-dataset/
we build the recommender. Step one is to compile the average ratings by title. 

In [24]:

Average_ratings = pd.DataFrame(df_movie_all.groupby('title')['rating'].mean())

In [26]:
Average_ratings.head(10)


,rating
title,
(500) Days of Summer (2009),3.666667
...And Justice for All (1979),3.166667
10 Cloverfield Lane (2016),3.678571
10 Things I Hate About You (1999),3.527778
101 Dalmatians (1996),3.074468
101 Dalmatians (One Hundred and One Dalmatians) (1961),3.431818
"11'09""01 - September 11 (2002)",4.000000
12 Angry Men (1957),4.149123
127 Hours (2010),3.833333


We now group that by total ratings as that can influence results

In [28]:
Average_ratings['Total Ratings'] = pd.DataFrame(df_movie_all.groupby('title')['rating'].count())

In [29]:
Average_ratings.head(10)

,rating,Total Ratings
title,,
(500) Days of Summer (2009),3.666667,336
...And Justice for All (1979),3.166667,3
10 Cloverfield Lane (2016),3.678571,28
10 Things I Hate About You (1999),3.527778,54
101 Dalmatians (1996),3.074468,94
101 Dalmatians (One Hundred and One Dalmatians) (1961),3.431818,44
"11'09""01 - September 11 (2002)",4.000000,2
12 Angry Men (1957),4.149123,513
127 Hours (2010),3.833333,18


The way this recommender works is by "if similar users rated this movie_X highly and rated movie_Y highly it is probable other fans of movie_X will like movie_Y". Below is a code that takes every User_ID and what they ranked each movie. It then searches for correlations based on the input.

In [56]:
movie_user = df_movie_all.pivot_table(index = 'userId_y', columns = 'title', values = 'rating')

In [57]:
movie_user

title,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),"11'09""01 - September 11 (2002)",12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),...,Young Frankenstein (1974),Z (1969),Zack and Miri Make a Porno (2008),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zombieland (2009),Zoolander (2001),Zulu (1964),eXistenZ (1999)
userId_y,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.5


In [271]:
correlations = movie_user.corrwith(movie_user[new_list[0]['title'].item()], numeric_only = True) 

In [168]:
correlations.info()

<class 'pandas.core.series.Series'>
Index: 1554 entries, (500) Days of Summer (2009) to eXistenZ (1999)
Series name: None
Non-Null Count  Dtype  
--------------  -----  
1232 non-null   float64
dtypes: float64(1)
memory usage: 56.6+ KB


In [60]:
correlations.head()

title
(500) Days of Summer (2009)          0.353833
...And Justice for All (1979)             NaN
10 Cloverfield Lane (2016)          -0.285732
10 Things I Hate About You (1999)    0.322741
101 Dalmatians (1996)                0.580479
dtype: float64

In [61]:
recommendation = pd.DataFrame(correlations,columns=['Correlation'])


In [62]:
recommendation.dropna(inplace=True) 



In [63]:
recommendation = recommendation.join(Average_ratings['Total Ratings']) 

In [65]:
recommendation.head(10)

,Correlation,Total Ratings
title,,
(500) Days of Summer (2009),0.353833,336
10 Cloverfield Lane (2016),-0.285732,28
10 Things I Hate About You (1999),0.322741,54
101 Dalmatians (1996),0.580479,94
101 Dalmatians (One Hundred and One Dalmatians) (1961),0.399643,44
12 Angry Men (1957),0.321827,513
127 Hours (2010),0.450711,18
13 Going on 30 (2004),0.076205,21
2001: A Space Odyssey (1968),0.014566,4469


In [68]:
recc = recommendation[recommendation['Total Ratings'] > 100].sort_values('Correlation', ascending = False).reset_index()

In [69]:
recc.head(10)

,title,Correlation,Total Ratings
0,Toy Story (1995),1.000000,645
1,Avengers: Infinity War - Part I (2018),0.942264,195
2,Twilight (2008),0.769261,110
3,Friends with Benefits (2011),0.768301,180
4,Toy Story 2 (1999),0.699211,776
5,Whiplash (2014),0.659713,342
6,"Incredibles, The (2004)",0.643301,500
7,Logan (2017),0.641903,125
8,Finding Nemo (2003),0.618701,423
9,Mary and Max (2009),0.616316,130


# Linking it all together

In [255]:
#requests movie from user
def request():
    movie_request = str(input('Please enter name of movie'))
    return movie_request

In [224]:
#takes in movie and turns list of possible candidates for that movie
def search_for_title(movie_r):
    new_list = []
    for item in df_movie['title']:
        if movie_r.lower() in item.lower():
            new_list.append(df_movie[df_movie['title'] == item])
        else:
            continue
    return new_list

In [225]:
#Since 'Toy Story' will return 1-3 this allows user to identify which movie the user wanted or if the movie does not appear to select another one
def select_title(m_list):
    print(m_list)
        
    try:
        title_num = input('Please enter the number on the list where the movie is to confirm choice or enter any letter to request a new one')
        #print(title_num)
        int(title_num)
       

    except:
        select_title(request())
    finally:
        title_num = int(title_num)
        return m_list[title_num]['title'].item()


In [ ]:
#here is our correlation grid that we need to correlate our movies
movie_user = df_movie_all.pivot_table(index = 'userId_y', columns = 'title', values = 'rating')

In [235]:
Average_ratings

,rating,Total Ratings
title,,
(500) Days of Summer (2009),3.666667,336
...And Justice for All (1979),3.166667,3
10 Cloverfield Lane (2016),3.678571,28
10 Things I Hate About You (1999),3.527778,54
101 Dalmatians (1996),3.074468,94
...,...,...
Zero Dark Thirty (2012),4.107143,70
Zombieland (2009),3.877358,318
Zoolander (2001),3.509259,324


In [214]:
#this function seaches the correlation grid and returns the correlations needed to make recommendations
def return_correlation(movie_y):
    correlations = movie_user.corrwith(movie_user[movie_y], numeric_only = True) 
    return correlations

In [267]:
def return_recommendations(correlations):
   
    recommendation = pd.DataFrame(correlations,columns=['Correlation'])
    recommendation.dropna(inplace=True)
    recommendation = recommendation.join(Average_ratings['Total Ratings'])
    recco2 = recommendation[recommendation['Total Ratings'] > 100].sort_values('Correlation', ascending = False).reset_index()
    return recco2

In [277]:
def recommend_movie():
    movie_title = request()
    movie_list = search_for_title(movie_title)
    movie_final = select_title(movie_list)
    return return_recommendations(return_correlation(movie_final))
    
    

In [222]:
new_list[0]['title'].item()

'Shawshank Redemption, The (1994)'

In [278]:
recommend_movie()

Please enter name of movie toy story


[   movieId             title                                       genres
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy,       movieId               title                                       genres
2355     3114  Toy Story 2 (1999)  Adventure|Animation|Children|Comedy|Fantasy,       movieId               title  \
7355    78499  Toy Story 3 (2010)   

                                                genres  
7355  Adventure|Animation|Children|Comedy|Fantasy|IMAX  ]


Please enter the number on the list where the movie is to confirm choice or enter any letter to request a new one 0


,title,Correlation,Total Ratings
0,Toy Story (1995),1.000000,645
1,Avengers: Infinity War - Part I (2018),0.942264,195
2,Twilight (2008),0.769261,110
3,Friends with Benefits (2011),0.768301,180
4,Toy Story 2 (1999),0.699211,776
...,...,...,...
283,Corpse Bride (2005),-0.213902,308
284,Terminator Salvation (2009),-0.226805,216
285,Transformers: Revenge of the Fallen (2009),-0.249944,140
286,"Girl, Interrupted (1999)",-0.376130,168
